In [1]:
# Projeto de Ciência de Dados – Predição de Churn em Telecomunicações

## 1. Contexto do Projeto

Neste projeto, eu desenvolvi uma solução de Ciência de Dados com o objetivo de prever o **churn de clientes** em uma empresa do setor de telecomunicações.  
O churn representa o cancelamento do serviço por parte do cliente e é um dos principais desafios estratégicos para empresas desse segmento, pois impacta diretamente a receita e os custos de aquisição de novos clientes.

Empresas de telecom lidam com alta competitividade e margens cada vez mais pressionadas. Nesse contexto, **reter clientes é significativamente mais barato do que adquirir novos**, tornando a previsão de churn uma ferramenta essencial para apoiar decisões de negócio baseadas em dados.

---

## 2. Problema de Negócio

O problema que busquei resolver foi:

> **Como identificar, com antecedência, clientes com maior probabilidade de cancelar o serviço?**

Ao prever o churn, a empresa pode:
- Criar campanhas de retenção mais assertivas
- Priorizar clientes de alto risco
- Reduzir perdas financeiras
- Melhorar a experiência do cliente

---

## 3. Objetivo do Projeto

O objetivo principal deste projeto foi **construir um modelo de machine learning capaz de prever se um cliente irá ou não entrar em churn**, com base em suas características contratuais, de uso e de perfil.

Os objetivos específicos incluem:
- Analisar o comportamento dos clientes
- Identificar padrões associados ao churn
- Preparar os dados adequadamente para modelagem
- Treinar e avaliar um modelo preditivo
- Apresentar visualizações claras dos resultados obtidos

---

## 4. Base de Dados Utilizada

Para o desenvolvimento do projeto, utilizei a base de dados **Churn Telecom**, que contém informações sobre clientes de uma empresa de telecomunicações, incluindo:
- Dados demográficos
- Informações de contrato
- Serviços contratados
- Histórico de cobrança
- Indicador de churn

Essa base é amplamente utilizada em estudos de churn e permite aplicar conceitos fundamentais de análise exploratória, pré-processamento e modelagem preditiva.

---

## 5. Abordagem Metodológica

O projeto foi desenvolvido seguindo uma abordagem estruturada, composta pelas seguintes etapas:

1. Entendimento do problema e do negócio  
2. Análise exploratória dos dados  
3. Tratamento e preparação dos dados  
4. Construção do modelo de machine learning  
5. Avaliação dos resultados  
6. Visualização e interpretação dos insights  
7. Conclusões e recomendações finais

Cada etapa foi documentada de forma clara, com foco na interpretação dos dados e na justificativa das decisões tomadas ao longo do processo.

SyntaxError: invalid syntax (82538336.py, line 5)

In [ ]:
# 1. Coleta e Carregamento dos Dados

Nesta etapa, eu carreguei a base `CHURN_TELECON.csv` e fiz uma inspeção inicial para entender:
- o tamanho do dataset (linhas e colunas),
- os tipos de dados,
- a presença de valores ausentes,
- possíveis inconsistências de categorização.

Meu objetivo aqui foi validar a qualidade do dado antes de qualquer análise mais profunda ou modelagem, seguindo o princípio de que a qualidade da entrada impacta diretamente a qualidade do modelo.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Ajuste o caminho se necessário
PATH = "CHURN_TELECON.csv"

# Importante: essa base usa ';' como separador
df = pd.read_csv(PATH, sep=";")

print("Shape:", df.shape)
display(df.head())

In [ ]:
# Tipos das colunas
display(df.dtypes)

# Checando duplicatas
print("Linhas duplicadas (linha inteira):", df.duplicated().sum())
print("customerID duplicado:", df["customerID"].duplicated().sum())

# Missing por coluna
missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)

diagnostico_missing = pd.DataFrame({
    "missing_count": missing,
    "missing_%": missing_pct.round(2)
})

display(diagnostico_missing)

In [ ]:
# Distribuição do target (incluindo NaN)
display(df["Churn"].value_counts(dropna=False))

# Plot simples da distribuição do churn (sem NaN)
churn_counts = df["Churn"].dropna().value_counts()
plt.figure()
plt.bar(churn_counts.index.astype(str), churn_counts.values)
plt.title("Distribuição do Target (Churn)")
plt.xlabel("Churn")
plt.ylabel("Quantidade")
plt.show()

print("Proporção (%):")
display((df["Churn"].dropna().value_counts(normalize=True) * 100).round(2))

In [ ]:
# 2. Principais Problemas de Qualidade e Estratégia de Tratamento

Nesta base, eu identifiquei três pontos importantes de qualidade:

1. **Valores ausentes**: existem colunas com valores faltantes, incluindo o próprio target (`Churn`) em poucas linhas.
2. **Inconsistência em categorias**: a variável `Genero` apresenta variações como `Male`, `M`, `f`, etc., o que exige padronização.
3. **Campos vazios correlacionados**: observei casos em que `PhoneService` e `Pagamento_Mensal` aparecem vazios simultaneamente.  
   Para evitar imputação arbitrária, eu usei uma estratégia guiada por regra: quando possível, estimei o pagamento mensal a partir de `Total_Pago` e `Tempo_como_Cliente`.

Meu objetivo foi produzir um dataset consistente e pronto para modelagem, minimizando distorções e mantendo coerência de negócio.


In [ ]:
df_clean = df.copy()

# 1) Remover linhas sem target (não dá para treinar sem rótulo)
df_clean = df_clean.dropna(subset=["Churn"]).reset_index(drop=True)

# 2) Padronizar Genero
# - Normaliza strings (strip/lower)
# - Mapeia variações
def padroniza_genero(x):
    if pd.isna(x):
        return "Unknown"
    x = str(x).strip().lower()
    if x in ["male", "m"]:
        return "Male"
    if x in ["female", "f"]:
        return "Female"
    return "Unknown"

df_clean["Genero"] = df_clean["Genero"].apply(padroniza_genero)

# 3) PhoneService: missing vira categoria explícita
df_clean["PhoneService"] = df_clean["PhoneService"].fillna("Unknown")

# 4) Pagamento_Mensal: imputação guiada por regra
# Se Pagamento_Mensal é NaN e temos Total_Pago e Tempo_como_Cliente, estimamos:
# Pagamento_Mensal_estimado = Total_Pago / max(Tempo_como_Cliente, 1)
mask_pm = df_clean["Pagamento_Mensal"].isna()

df_clean.loc[mask_pm, "Pagamento_Mensal"] = (
    df_clean.loc[mask_pm, "Total_Pago"] / df_clean.loc[mask_pm, "Tempo_como_Cliente"].clip(lower=1)
)

# Se ainda sobrar NaN (caso extremo), preenche com mediana
df_clean["Pagamento_Mensal"] = df_clean["Pagamento_Mensal"].fillna(df_clean["Pagamento_Mensal"].median())

# 5) Checagem final de missing
display(df_clean.isna().sum().sort_values(ascending=False))
print("Shape final:", df_clean.shape)

display(df_clean.head())

In [ ]:
# Checando se não sobrou churn nulo
print("Churn nulo após tratamento:", df_clean["Churn"].isna().sum())

# Valores únicos para conferir padronização do Genero e PhoneService
print("Genero - valores únicos:", df_clean["Genero"].unique())
print("PhoneService - valores únicos:", df_clean["PhoneService"].unique())

# Estatísticas rápidas dos numéricos
display(df_clean[["Tempo_como_Cliente", "Pagamento_Mensal", "Total_Pago", "Idoso"]].describe())

In [ ]:
# 3. Análise Exploratória de Dados (EDA)

Nesta etapa, eu realizei uma análise exploratória dos dados com o objetivo de compreender o comportamento dos clientes e identificar padrões associados ao churn.

A análise exploratória é fundamental para:
- validar hipóteses de negócio,
- identificar relações relevantes entre variáveis,
- detectar possíveis problemas de desbalanceamento,
- e orientar decisões futuras de modelagem e feature engineering.

Todas as análises foram feitas considerando o dataset já tratado, garantindo consistência e evitando vieses decorrentes de problemas de qualidade dos dados.

In [ ]:
# Distribuição do target
churn_dist = df_clean["Churn"].value_counts()
churn_pct = df_clean["Churn"].value_counts(normalize=True) * 100

display(pd.DataFrame({
    "Quantidade": churn_dist,
    "Percentual (%)": churn_pct.round(2)
}))

plt.figure()
plt.bar(churn_dist.index.astype(str), churn_dist.values)
plt.title("Distribuição do Churn")
plt.xlabel("Churn")
plt.ylabel("Quantidade de Clientes")
plt.show()

In [ ]:
### 3.1 Distribuição do Churn

A análise da variável alvo mostrou que a base apresenta um **desbalanceamento moderado**, com a maioria dos clientes não cancelando o serviço.

Esse ponto é relevante, pois modelos treinados sem atenção a esse desbalanceamento podem aprender a favorecer a classe majoritária, reduzindo a capacidade de identificar corretamente clientes em risco de churn.


In [ ]:
# Taxa média de churn por tipo de contrato
churn_contrato = (
    df_clean
    .groupby("Tipo_Contrato")["Churn_bin"]
    .mean()
    .reset_index()
)

display(churn_contrato)

plt.figure()
plt.bar(churn_contrato["Tipo_Contrato"], churn_contrato["Churn_bin"])
plt.title("Taxa de Churn por Tipo de Contrato")
plt.ylabel("Taxa de Churn")
plt.xticks(rotation=45)
plt.show()

In [ ]:
### 3.2 Churn por Tipo de Contrato

Ao analisar a taxa média de churn por tipo de contrato, observei que clientes com **contratos mensais** apresentam uma probabilidade de cancelamento significativamente maior do que aqueles com contratos de maior duração.

Esse resultado está alinhado com a lógica de negócio, pois contratos mais longos criam maior compromisso do cliente com a empresa, reduzindo a propensão ao churn.


In [ ]:
# Binning do tempo como cliente
df_clean["Faixa_Tempo"] = pd.cut(
    df_clean["Tempo_como_Cliente"],
    bins=[0, 6, 12, 24, 48, 72],
    include_lowest=True
)

churn_tempo = (
    df_clean
    .groupby("Faixa_Tempo")["Churn_bin"]
    .mean()
    .reset_index()
)

display(churn_tempo)

plt.figure()
plt.plot(churn_tempo["Faixa_Tempo"].astype(str), churn_tempo["Churn_bin"], marker="o")
plt.title("Taxa de Churn por Tempo como Cliente")
plt.xlabel("Tempo como Cliente (meses)")
plt.ylabel("Taxa de Churn")
plt.xticks(rotation=45)
plt.show()

In [ ]:
Durante a análise, eu identifiquei que a variável `Churn` estava em formato categórico (texto), o que impedia o cálculo de médias por grupo.  
Para resolver isso, eu criei uma variável auxiliar `Churn_bin`, mapeando churn para valores binários (0 e 1).  

Essa abordagem é estatisticamente adequada porque a **média de uma variável binária** representa diretamente a **taxa/probabilidade empírica** de churn em cada grupo analisado.


In [ ]:
# Criação de faixas de pagamento mensal
df_clean["Faixa_Pagamento"] = pd.qcut(
    df_clean["Pagamento_Mensal"],
    q=4,
    duplicates="drop"
)

churn_pagamento = (
    df_clean
    .groupby("Faixa_Pagamento")["Churn_bin"]
    .mean()
    .reset_index()
)

display(churn_pagamento)

plt.figure()
plt.plot(
    churn_pagamento["Faixa_Pagamento"].astype(str),
    churn_pagamento["Churn_bin"],
    marker="o"
)
plt.title("Taxa de Churn por Faixa de Pagamento Mensal")
plt.xlabel("Faixa de Pagamento Mensal")
plt.ylabel("Taxa de Churn")
plt.xticks(rotation=45)
plt.show()

In [ ]:
### 3.3 Churn e Pagamento Mensal

A análise mostrou que clientes que pagam valores mensais mais elevados apresentam uma taxa de churn maior.

Esse comportamento pode estar associado à sensibilidade a preço ou à percepção de que o custo do serviço não está alinhado ao valor entregue, indicando uma oportunidade de ação para estratégias de retenção focadas nesse perfil de cliente.

In [ ]:
# 4. Pré-processamento para Modelagem

Nesta etapa, eu preparei os dados para a etapa de modelagem preditiva.  
O foco foi transformar as variáveis em um formato adequado para algoritmos de machine learning, garantindo que o processo fosse feito de maneira organizada e sem vazamento de informação.

As decisões de pré-processamento foram tomadas considerando:
- o tipo das variáveis (numéricas e categóricas),
- a interpretabilidade do modelo,
- e a necessidade de manter consistência entre treino e teste.

In [ ]:
# Target final (binário)
y = df_clean["Churn_bin"]

# Removendo colunas que não devem entrar no modelo
colunas_excluir = [
    "Churn",        # versão textual do target
    "Churn_bin",    # target (não entra como feature)
    "customerID",   # identificador
    "Faixa_Tempo",  # criada apenas para EDA
    "Faixa_Pagamento"
]

X = df_clean.drop(columns=colunas_excluir, errors="ignore")

print("Shape X:", X.shape)
print("Shape y:", y.shape)

In [ ]:
Antes do encoding, eu analisei os tipos de variáveis para separar corretamente variáveis numéricas e categóricas.  
Essa separação é importante porque cada tipo exige um tratamento específico durante o pré-processamento.

In [ ]:
# Variáveis numéricas e categóricas
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X.select_dtypes(include=["object"]).columns.tolist()

print("Numéricas:", num_features)
print("Categóricas:", cat_features)

In [ ]:
Para as variáveis categóricas, optei por utilizar **One-Hot Encoding**, pois:
- a base possui cardinalidade baixa a moderada,
- os modelos escolhidos conseguem lidar bem com esse tipo de codificação,
- e essa abordagem facilita a interpretação dos resultados.

As variáveis numéricas foram mantidas em sua escala original nesta etapa.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print("Treino:", X_train.shape, "Teste:", X_test.shape)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(
    drop="first",
    handle_unknown="ignore",
    sparse_output=False
)

# Fit apenas no treino
X_train_cat = encoder.fit_transform(X_train[cat_features])
X_test_cat = encoder.transform(X_test[cat_features])

# DataFrames codificados
X_train_cat = pd.DataFrame(
    X_train_cat,
    columns=encoder.get_feature_names_out(cat_features),
    index=X_train.index
)

X_test_cat = pd.DataFrame(
    X_test_cat,
    columns=encoder.get_feature_names_out(cat_features),
    index=X_test.index
)

# Numéricos
X_train_num = X_train[num_features]
X_test_num = X_test[num_features]

# Dataset final
X_train_final = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

print("Shape final treino:", X_train_final.shape)
print("Shape final teste:", X_test_final.shape)

In [ ]:
Nota técnica: devido à versão do scikit-learn utilizada, o parâmetro `sparse` do OneHotEncoder foi substituído por `sparse_output`. O ajuste foi feito para garantir compatibilidade com versões mais recentes da biblioteca.

In [ ]:
## 4. Pré-processamento para Modelagem

Nesta etapa, eu preparei os dados para a construção dos modelos de machine learning.  
O objetivo foi transformar as variáveis em um formato adequado para os algoritmos, garantindo consistência, reprodutibilidade e evitando vazamento de informações entre os conjuntos de treino e teste.

Inicialmente, defini a variável alvo (`Churn`) em formato binário, criando a coluna `Churn_bin`, que representa se o cliente cancelou (1) ou não (0) o serviço. Essa transformação foi essencial para permitir o uso de métricas e modelos de classificação.

Em seguida, removi colunas que não deveriam entrar no modelo, como identificadores únicos e variáveis criadas exclusivamente para análise exploratória, evitando que informações irrelevantes ou derivadas influenciassem o treinamento.

As variáveis explicativas foram então separadas em **numéricas** e **categóricas**, pois cada tipo exige um tratamento específico.  
Para as variáveis categóricas, utilizei **One-Hot Encoding**, uma técnica amplamente empregada em problemas de classificação e adequada para variáveis com baixa a média cardinalidade. O processo de encoding foi ajustado para a versão do scikit-learn utilizada, garantindo compatibilidade técnica.

A separação dos dados em conjuntos de treino e teste foi realizada de forma **estratificada**, preservando a proporção da variável alvo e assegurando uma avaliação mais confiável do modelo.

Ao final desta etapa, obtive conjuntos de dados de treino e teste totalmente preparados para a etapa de modelagem, com todas as variáveis em formato numérico e alinhadas entre si, permitindo o treinamento de modelos de machine learning de forma segura e eficiente.

In [ ]:
# 5. Modelagem Preditiva

Nesta etapa, eu desenvolvi modelos de machine learning com o objetivo de prever o churn dos clientes.  
A modelagem foi iniciada com um modelo baseline simples, utilizado como referência mínima de desempenho.

Em seguida, construí um modelo de classificação mais robusto, buscando equilibrar desempenho preditivo e interpretabilidade, características importantes para aplicações reais de negócio.


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Baseline: sempre prever a classe majoritária
baseline = DummyClassifier(strategy="most_frequent", random_state=42)
baseline.fit(X_train_final, y_train)

y_pred_baseline = baseline.predict(X_test_final)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nClassification Report - Baseline")
print(classification_report(y_test, y_pred_baseline))

confusion_matrix(y_test, y_pred_baseline)

In [ ]:
O modelo baseline, que sempre prevê a classe majoritária, apresentou uma acurácia elevada devido ao desbalanceamento da base.  
No entanto, ele falha completamente em identificar clientes que realmente entram em churn, o que o torna inadequado para uso prático.

Esse resultado reforça a necessidade de um modelo mais sofisticado, capaz de capturar padrões associados ao cancelamento.

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",  # importante para churn
    random_state=42
)

log_reg.fit(X_train_final, y_train)

y_pred = log_reg.predict(X_test_final)
y_proba = log_reg.predict_proba(X_test_final)[:, 1]

print("Classification Report - Logistic Regression")
print(classification_report(y_test, y_pred))

confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC:", round(roc_auc, 3))

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - Regressão Logística")
plt.legend()
plt.show()

In [ ]:
O modelo de Regressão Logística apresentou desempenho significativamente superior ao baseline, especialmente na identificação de clientes propensos ao churn.

A métrica ROC-AUC indica que o modelo possui boa capacidade de separação entre clientes que cancelam e os que permanecem ativos, mesmo considerando o desbalanceamento da base.

Além disso, a Regressão Logística oferece interpretabilidade, permitindo entender como diferentes variáveis influenciam a probabilidade de churn.


In [ ]:
# Coeficientes do modelo
coeficientes = pd.DataFrame({
    "Feature": X_train_final.columns,
    "Coeficiente": log_reg.coef_[0]
}).sort_values(by="Coeficiente", ascending=False)

display(coeficientes.head(10))
display(coeficientes.tail(10))

In [ ]:
A análise dos coeficientes do modelo mostrou que variáveis relacionadas a tipo de contrato, tempo como cliente e valor do pagamento mensal possuem forte influência na probabilidade de churn.

Esses resultados estão alinhados com os insights obtidos na análise exploratória, reforçando a consistência do modelo e sua aplicabilidade para apoiar decisões estratégicas de retenção de clientes.


In [ ]:
# 6. Resultados e Avaliação do Modelo

Nesta etapa, eu consolidei os principais resultados obtidos com o modelo de machine learning desenvolvido para predição de churn.

A avaliação foi realizada utilizando métricas adequadas para problemas de classificação com classes desbalanceadas, com foco especial na capacidade do modelo de identificar clientes propensos ao cancelamento.


In [ ]:
## 6.1 Desempenho do Modelo

O modelo de Regressão Logística apresentou desempenho significativamente superior ao modelo baseline, demonstrando capacidade real de identificar padrões associados ao churn.

Os principais destaques foram:
- Desempenho consistente na separação entre clientes que cancelam e os que permanecem ativos, conforme evidenciado pela curva ROC.
- Melhora substancial na identificação da classe minoritária (churn), quando comparado ao baseline.
- Resultados alinhados com os insights observados durante a análise exploratória.

Esses resultados indicam que o modelo é adequado como ponto de partida para apoiar estratégias de retenção de clientes.

In [ ]:
## 6.2 Visualização dos Resultados

Para facilitar a interpretação dos resultados, utilizei visualizações gráficas que permitem avaliar o desempenho do modelo e os fatores mais relevantes para a previsão de churn.

As principais visualizações apresentadas foram:
- Curva ROC, demonstrando a capacidade de discriminação do modelo.
- Matriz de confusão, permitindo avaliar acertos e erros de classificação.
- Análise dos coeficientes da Regressão Logística, destacando as variáveis com maior impacto na probabilidade de churn.

Essas visualizações tornam os resultados mais acessíveis e facilitam a comunicação com stakeholders técnicos e de negócio.

In [ ]:
# 7. Conclusões

Ao longo deste projeto, eu desenvolvi uma solução de Ciência de Dados voltada para a predição de churn em uma empresa de telecomunicações.

A análise exploratória revelou que o churn está fortemente associado a fatores como:
- tipo de contrato,
- tempo como cliente,
- e valor do pagamento mensal.

O modelo de machine learning desenvolvido foi capaz de capturar esses padrões e apresentou desempenho superior ao baseline, demonstrando potencial para apoiar decisões estratégicas de retenção de clientes.

Apesar dos bons resultados, este projeto representa uma primeira abordagem. Modelos mais complexos e dados adicionais podem contribuir para ganhos adicionais de performance.


In [ ]:
## 7.1 Próximos Passos

Como continuidade deste projeto, algumas melhorias possíveis incluem:
- Testar modelos mais avançados, como Random Forest ou Gradient Boosting.
- Ajustar hiperparâmetros para otimização de desempenho.
- Avaliar técnicas de balanceamento de classes.
- Incorporar dados comportamentais mais detalhados.
- Implementar o modelo em ambiente produtivo para monitoramento contínuo de churn.